In [1]:
import pandas as pd
from numpy.linalg import norm
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit

In [ ]:
movie_genre = pd.read_csv('../hw3_data/Movielens/movie_genre.dat', sep='\t', header=None)
movie_movie = pd.read_csv('../hw3_data/Movielens/movie_movie(knn).dat', sep='\t', header=None)
user_age = pd.read_csv('../hw3_data/Movielens/user_age.dat', sep='\t', header=None)
user_movie = pd.read_csv('../hw3_data/Movielens/user_movie.dat', sep='\t', header=None)
user_occupation = pd.read_csv('../hw3_data/Movielens/user_occupation.dat', sep='\t', header=None)
user_user = pd.read_csv('../hw3_data/Movielens/user_user(knn).dat', sep='\t', header=None)

In [ ]:
user_interaction = user_movie[1].value_counts()
user_movie = user_movie[user_movie[1].isin(user_interaction.index[user_interaction.gt(3)])]
user_movie.columns = ['user', 'movie', 'rating', 'timestamp']

In [ ]:
movie_genre = np.load('../hw3_data/Movielens/movie_genre.npy')
movie_movie = np.load('../hw3_data/Movielens/movie_movie.npy')
user_age = np.load('../hw3_data/Movielens/user_age.npy')
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')
user_occupation = np.load('../hw3_data/Movielens/user_occupation.npy')
user_user = np.load('../hw3_data/Movielens/user_user.npy')

X = user_movie[:,:2]
y = user_movie[:,-1]
entity_user = 943
entity_item = 1682
age, occupation, genre = 8, 21, 18 # entity amount 

In [ ]:
def get_embedding(data, ):
    user, item = data
    
    ua = user_age[user_age[:,0] == user][:,1][0]
    uo = user_occupation[user_occupation[:,0] == user][:,1][0]
    mg = movie_genre[movie_genre[:,0] == item][:,1]
    mg = "|".join(str(x) for x in mg)
    if mg =='':
        mg = str(0)
    # fea_genre = np.zeros((1, genre), dtype=int)
    # mg = movie_genre[movie_genre[:,0] == item][:,1]
    # fea_genre[0, mg-1] = 1

    # user_knn = np.zeros((1, entity_user), dtype=int)
    # uu = user_user[user_user[:,0] == user][:,1].astype(int)
    # user_knn[0, uu-1] = 1

    # item_knn = np.zeros((1, entity_item), dtype=int)
    # mm = movie_movie[movie_movie[:,0] == item][:,1].astype(int)
    # item_knn[0, mm-1] = 1
    
    # train_example = np.concatenate((user_index, item_index, fea_age, fea_occupation, fea_genre, user_knn, item_knn), axis=1)
    train_example = np.array([user,item,ua,uo,mg])
    return train_example


embed_X = np.apply_along_axis(get_embedding, 1, X)
embed = np.concatenate((embed_X, y.reshape(-1,1)),axis =1)

In [ ]:
embed = pd.DataFrame(embed, columns=['user', 'movie', 'age', 'occupation', 'genre','rating'])
embed

In [ ]:
embed.to_csv('../hw3_data/Movielens/movielens_sample.csv',index=False)

# Yelp

In [46]:
business_category = np.load('../hw3_data/Yelp/business_category.npy')
business_city = np.load('../hw3_data/Yelp/business_city.npy')
user_business = np.load('../hw3_data/Yelp/user_business.npy')
user_compliment = np.load('../hw3_data/Yelp/user_compliment.npy')
user_user = np.load('../hw3_data/Yelp/user_user.npy')

X = user_business[:,:2]
y = user_business[:,-1]
entity_user = 16239
entity_item = 14284
compliment, category, city = 11, 511, 47 # entity amount 

In [50]:
def get_embedding(data, ):
    user, item = data
    
    # print(user, item)
    bc = business_city[business_city[:,0] == item][:,1]
    if len(bc) == 0:
        bc = 0
    else:
        bc = bc[0]
    bcate = business_category[business_category[:,0] == item][:,1]
    bcate = "|".join(str(x) for x in bcate)
    if bcate =='':
        bcate = str(0)
    uc = user_compliment[user_compliment[:,0] == user][:,1]
    uc = "|".join(str(x) for x in uc)
    if uc =='':
        uc = str(0)
    train_example = np.array([user, item, bc, bcate, uc])
    return train_example


embed_X = np.apply_along_axis(get_embedding, 1, X)
embed = np.concatenate((embed_X, y.reshape(-1,1)),axis =1)

In [52]:
embed = pd.DataFrame(embed, columns=['user', 'business', 'city', 'category', 'compliment','rating'])
embed

,user,business,city,category,compliment,rating
0,1,8391,31,45|200|328,0,5
1,1,8971,35,45|200|328,0,5
2,2,186,31,187|306|407,1|2|3|4|5|6|7|8|9|11,5
3,2,205,31,407|435,1|2|3|4|5|6|7|8|9|11,5
4,2,209,31,45|76|328|407,1|2|3|4|5|6|7|8|9|11,4
...,...,...,...,...,...,...
185385,16238,13256,26,68|407,7|9|11,5
185386,16239,2580,35,16|45|328|407,0,4
185387,16239,10525,35,45|306|328|407|443,0,1
185388,16239,12714,35,16|45|65|328|407,0,4


In [53]:
embed.to_csv('../hw3_data/Yelp/Yelp_sample.csv',index=False)

# book

In [2]:
book_author = np.load('../hw3_data/Douban_Book/book_author.npy')
book_publisher = np.load('../hw3_data/Douban_Book/book_publisher.npy')
book_year = np.load('../hw3_data/Douban_Book/book_year.npy')
user_book = np.load('../hw3_data/Douban_Book/user_book.npy')
user_group = np.load('../hw3_data/Douban_Book/user_group.npy')
user_location = np.load('../hw3_data/Douban_Book/user_location.npy')
user_user = np.load('../hw3_data/Douban_Book/user_user.npy')



X = user_book[:,:2]
y = user_book[:,-1]
entity_user = 13024
entity_item = 22347
group, location, author, publisher, year  = 2936, 38, 10805, 1815, 64  # entity amount 

In [ ]:
def get_embedding(data, ):
    user, item = data
    
    # print(user, item)
    ba = book_author[book_author[:,0] == item][:,1]
    if len(ba) == 0:
        ba = 0
    else:
        ba = ba[0]
    bp = book_publisher[book_publisher[:,0] == item][:,1]
    if len(bp) == 0:
        bp = 0
    else:
        bp = bp[0]
    by = book_year[book_year[:,0] == item][:,1]
    if len(by) == 0:
        by = 0
    else:
        by = by[0]
    
    ug = user_group[user_group[:,0] == user][:,1]
    ug = "|".join(str(x) for x in ug)
    if ug =='':
        ug = str(0)
    ul = user_location[user_location[:,0] == user][:,1]
    if len(ul) == 0:
        ul = 0
    else:
        ul = ul[0]
    # uu = user_user[user_user[:,0] == 2000][:,1]
    # uu = "|".join(str(x) for x in uu)
    # if uu =='':
    #     uu = str(0)

    train_example = np.array([user, item, ba, bp, by, ug, ul])
    return train_example


embed_X = np.apply_along_axis(get_embedding, 1, X)
embed = np.concatenate((embed_X, y.reshape(-1,1)),axis =1)
embed = pd.DataFrame(embed, columns=['user', 'book', 'author', 'publisher', 'year', 'group', 'location', 'user' 'rating'])
embed.to_csv('../hw3_data/Douban_Book/Douban_Book_sample.csv',index=False)

In [ ]:
embed = pd.DataFrame(embed, columns=['user', 'book', 'author', 'publisher', 'year', 'group', 'location', 'user' 'rating'])
embed.to_csv('../hw3_data/Douban_Book/Douban_Book_sample.csv',index=False)